In [4]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import time

In [5]:
paths = ['dataset/']

In [6]:
def imshow(img):
    fig, ax = plt.subplots(1,1, figsize=(8,8))
    ax.imshow(img, cmap="gray")

In [7]:
possible_labels = os.listdir(os.path.join(paths[0]))
possible_labels.pop(0)
possible_labels

['closed', 'open']

In [8]:
data = []
for i, path in enumerate(paths):
    for label_int, label_string in enumerate(possible_labels):
        filenames = glob.glob(path + label_string + '/*.jpg')
        for filename in filenames:
            img = cv.imread(filename, cv.IMREAD_GRAYSCALE)
            data.append([label_int, cv.resize(img, (224,224))])
print(len(data))

2679


In [9]:
import random
random.Random(0).shuffle(data)

x_train = []
y_train = []
x_test = []
y_test = []

for i, sample in enumerate(data):
    label, img = sample[0], sample[1]
    if i <= 0.7 * len(data):
        x_train.append(img)
        y_train.append(label)
    else:
        x_test.append(img)
        y_test.append(label)

x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

In [10]:
x_train = x_train/255
x_test = x_test/255

In [11]:
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

In [12]:
print(x_train.shape)
print(x_test.shape)

(1876, 224, 224)
(803, 224, 224)


In [13]:
print(y_train.shape)
print(y_test.shape)

(1876, 2)
(803, 2)


# Red neuronal

In [20]:
def model():
    model = Sequential()
    model.add(Conv2D(32,(2,2), input_shape=(224, 224, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(3,3)))
    model.add(Conv2D(64,(2,2), input_shape=(224, 224, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(128,(2,2), input_shape=(224, 224, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(2, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.AUC(name='auc'), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    return model

In [21]:
model = model()
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 223, 223, 32)   │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 73, 73, 64)     │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 35, 35, 128)    │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │     2,367,552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,408,994 (9.19 MB)

 Trainable params: 2,408,994 (9.19 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
model.fit(x_train, y_train, epochs=10, batch_size=64)

Epoch 1/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 8s 251ms/step - accuracy: 0.5593 - auc: 0.5912 - auc_3: 0.5912 - loss: 0.6790 - precision: 0.5655 - recall: 0.5507
Epoch 2/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 7s 243ms/step - accuracy: 0.8849 - auc: 0.9250 - auc_3: 0.9250 - loss: 0.3628 - precision: 0.8455 - recall: 0.8402
Epoch 3/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 7s 246ms/step - accuracy: 0.9708 - auc: 0.9945 - auc_3: 0.9945 - loss: 0.1017 - precision: 0.9712 - recall: 0.9566
Epoch 4/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 8s 250ms/step - accuracy: 0.9919 - auc: 0.9991 - auc_3: 0.9991 - loss: 0.0369 - precision: 0.9894 - recall: 0.9921
Epoch 5/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 8s 253ms/step - accuracy: 0.9935 - auc: 0.9996 - auc_3: 0.9996 - loss: 0.0245 - precision: 0.9923 - recall: 0.9920
Epoch 6/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 8s 260ms/step - accuracy: 0.9957 - auc: 0.9995 - auc_3: 0.9995 - loss: 0.0194 - precision: 0.9957 - recall: 0.9934
Epoch 7/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 8s 262ms/step - accuracy: 0.9982 - auc: 1.00

In [23]:
model.evaluate(x_test, y_test)

26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.9934 - auc: 0.9977 - auc_3: 0.9977 - loss: 0.0278 - precision: 0.9979 - recall: 0.9932


[0.015332956798374653,
 0.9975093603134155,
 0.9986979961395264,
 0.9986979961395264,
 0.9987515807151794,
 0.9962639808654785]

In [ ]:
p = 1
q = p + 1

for i in range(10):
    prediction = model.predict(x_test[p:q])
    prediction_label = prediction.argmax()
    print(f'Prediction: {prediction_label}')
    print(f'Etiqueta real: {y_test[p].argmax()}')
    p += 1
    q = p+1
    time.sleep(1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Prediction: 0
Etiqueta real: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Prediction: 0
Etiqueta real: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Prediction: 0
Etiqueta real: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Prediction: 1
Etiqueta real: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Prediction: 1
Etiqueta real: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Prediction: 0
Etiqueta real: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Prediction: 1
Etiqueta real: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Prediction: 1
Etiqueta real: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Prediction: 0
Etiqueta real: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Prediction: 0
Etiqueta real: 0


In [24]:
model.save("models/open-closed-eyes-v11.h5")